In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import fiona
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import numpy as np
import os
from engarde.decorators import none_missing, unique_index, is_shape
import engarde.generic
import engarde.decorators as ed
from pyproj import Geod
# for plotting map Leaflet library
import folium
import scipy
from itertools import cycle
import math
from scipy.spatial import distance
import timeit
import time

In [3]:
## ---- for mapboxgl Library
from mapboxgl.viz import *
from collections import defaultdict
#from mapboxgl.utils import df_to_geojson, create_radius_stops, scale_between, df_to_hexbin, hexbin
from mapboxgl.utils import df_to_geojson, create_radius_stops
#from mapboxgl.utils import hexbin
from mapboxgl.colors import create_color_stops


In [4]:
start_time = timeit.default_timer()

df_Broadcast = pq.read_table('ais/2011/01/Zone18_2011_01/Broadcast.parquet').to_pandas()
df_broadcast_month02 = pq.read_table('ais/2011/02/Zone18_2011_02/Broadcast.parquet').to_pandas()
df_broadcast_month03 = pq.read_table('ais/2011/03/Zone18_2011_03/Broadcast.parquet').to_pandas()


frames = [df_Broadcast, df_broadcast_month02, df_broadcast_month03]
result_frame = pd.concat(frames)

result_frame = result_frame.sort_values(['MMSI', 'BaseDateTime'])

result_frame['CheckMMSI'] = (result_frame['MMSI'] != result_frame['MMSI'].shift(1))


#shifting the whole dataframe rows and calculating radians latitude
result_frame['latRad'] = np.where(result_frame['CheckMMSI'] != True , result_frame['lat'].shift(1).apply(radians), np.nan)
result_frame['lonRad'] = np.where(result_frame['CheckMMSI'] != True , result_frame['lon'].shift(1).apply(radians), np.nan)

#taking radinas for the small values
lat1 = result_frame['lat'].apply(radians)
lon1 = result_frame['lon'].apply(radians)

#taking difference for latitude and longitude
dlon = result_frame['lonRad'] - lon1 
dlat = result_frame['latRad'] - lat1

#calculating te distance
harvesineDist = (dlat/2).apply(sin)**2 + lat1.apply(cos) * result_frame['latRad'].apply(cos) * (dlon/2).apply(sin)**2
c = 2* np.arcsin(harvesineDist.apply(sqrt))
dist = 6367 * c

##--- Creating a new Data frame for Distance
list_dist =  pd.DataFrame({"Distance": dist})
#concatinating the Dataframe with the old Dataframe
result_frame = pd.concat([result_frame, list_dist], axis=1)
result_frame


#chaning the datetime from string to datetime stamp
result_frame['DateTime'] = pd.DatetimeIndex(result_frame['BaseDateTime'])
#df_Broadcast['DateTime']
#shfting the date on bool value and taking out the difference of two dates
result_frame['DateTime2'] = np.where(result_frame['CheckMMSI'] != True , result_frame['DateTime'].shift(1), result_frame['DateTime'])
result_frame['time_diff'] = (result_frame['DateTime'] - result_frame['DateTime2']) 
#calculating the total hours from baseDateTime
time = pd.DatetimeIndex(result_frame['time_diff'])
result_frame['totalHours'] = time.hour + (time.minute/60) + (time.second/3600)
result_frame= result_frame.reset_index()
#df_Broadcast

result_frame['Speed'] = result_frame['Distance'] / result_frame['totalHours']
result_frame["Stopped"] = result_frame["Speed"].isnull() | (result_frame["Speed"] > 5)
result_frame["subVoyageIDs"] = ((result_frame["Stopped"].shift(1) != result_frame["Stopped"]) | (result_frame["Speed"] < 5)).astype(int).cumsum() 
result_frame

print (timeit.default_timer() - start_time)

258.5772450361401


In [5]:
result_frame

,index,BaseDateTime,COG,Heading,MMSI,ROT,ReceiverID,ReceiverType,SOG,Status,...,latRad,lonRad,Distance,DateTime,DateTime2,time_diff,totalHours,Speed,Stopped,subVoyageIDs
0,853,2011-01-01T00:01:00,0,511,0,128,003669730,b,0,5,...,NaN,NaN,NaN,2011-01-01 00:01:00,2011-01-01 00:01:00,00:00:00,0.000000,NaN,True,1
1,2165,2011-01-01T00:04:00,0,511,0,128,003669730,b,0,5,...,0.642927,-1.331281,0.002879,2011-01-01 00:04:00,2011-01-01 00:01:00,00:03:00,0.050000,0.057579,False,2
2,3438,2011-01-01T00:07:00,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331281,0.002232,2011-01-01 00:07:00,2011-01-01 00:04:00,00:03:00,0.050000,0.044646,False,3
3,4713,2011-01-01T00:10:00,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331281,0.002158,2011-01-01 00:10:00,2011-01-01 00:07:00,00:03:00,0.050000,0.043154,False,4
4,6054,2011-01-01T00:13:00,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331281,0.000720,2011-01-01 00:13:00,2011-01-01 00:10:00,00:03:00,0.050000,0.014403,False,5
5,7385,2011-01-01T00:16:00,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331281,0.001949,2011-01-01 00:16:00,2011-01-01 00:13:00,00:03:00,0.050000,0.038972,False,6
6,8734,2011-01-01T00:19:00,0,511,0,128,003669730,b,0,5,...,0.642927,-1.331281,0.000996,2011-01-01 00:19:00,2011-01-01 00:16:00,00:03:00,0.050000,0.019927,False,7
7,10045,2011-01-01T00:21:59,0,511,0,128,003669730,b,0,5,...,0.642927,-1.331281,0.001966,2011-01-01 00:21:59,2011-01-01 00:19:00,00:02:59,0.049722,0.039548,False,8
8,11374,2011-01-01T00:24:59,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331282,0.004485,2011-01-01 00:24:59,2011-01-01 00:21:59,00:03:00,0.050000,0.089709,False,9
9,12661,2011-01-01T00:27:59,0,511,0,128,003669730,b,0,5,...,0.642926,-1.331281,0.005141,2011-01-01 00:27:59,2011-01-01 00:24:59,00:03:00,0.050000,0.102816,False,10
